# initialize and upload cw20 token list into a hub-like contract
`cw20 token list` contract is experimental and is not a part of hub.

In [1]:
import pandas as pd
from warnings import filterwarnings
from tqdm.notebook import tqdm
import json
from ipfshttpclient import Client
from multiaddr import Multiaddr
from typing import Optional, Union
from re import sub

from cyber_sdk.client.lcd import LCDClient
from cyber_sdk.client.lcd.wallet import Wallet
from cyber_sdk.core import AccAddress
from cyber_sdk.key.mnemonic import MnemonicKey
from cyberutils.bash import get_json_from_bash_query
from cyberutils.contract import execute_contract

from ..src.contract_utils import query_contract, instantiate_contract
from ..config import NODE_RPC_URL, LCD_CLIENT, WALLET_SEED, IPFS_HOST

filterwarnings('ignore')

In [2]:
ipfs_client = Client(Multiaddr(IPFS_HOST))
mk = MnemonicKey(mnemonic=WALLET_SEED)
WALLET = LCD_CLIENT.wallet(mk)
WALLET_ADDRESS = WALLET.key.acc_address
INIT_CONTRACT = False
contract_address = 'bostrom1uay5ex5z547dw7mhw4puwcg998zu474czftlv5s9qpmywl2umaus4vd6x3'

## generate data

In [3]:
cw20_contracts = get_json_from_bash_query(
    bash_command=f'cyber query wasm list-contract-by-code 1 -o=json --limit=1000 --node={NODE_RPC_URL}')['contracts']

In [4]:
cw20_json = []
for cw20_contract in tqdm(cw20_contracts):
    cw20_metadata = \
        query_contract(contract_address=cw20_contract,
                       query='''{"token_info": {}}''')['data']
    cw20_marketing_info = \
        query_contract(contract_address=cw20_contract,
                       query='''{"marketing_info": {}}''')['data']
    logo = cw20_marketing_info['logo']
    cw20_json.append(
        {
            'chain_id': 'bostrom',
            'channel_id': '',
            'contract': cw20_contract,
            'decimals': str(cw20_metadata['decimals']),
            'logo': ipfs_client.add_str('' if logo is None or logo == {'url': ''} else str(logo['url'])),
            'ticker': sub(r'\W', '', string=cw20_metadata['symbol']).upper()
        }
    )
cw20_df = pd.DataFrame(cw20_json)
cw20_df

  0%|          | 0/106 [00:00<?, ?it/s]

,chain_id,channel_id,contract,decimals,logo,ticker
0,bostrom,,bostrom14hj2tavq8fpesdwxxcu44rty3hh90vhujrvcms...,0,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,FCWT
1,bostrom,,bostrom1suhgf5svhu4usrurvxzlgn54ksxmn8gljarjtx...,0,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,NTN
2,bostrom,,bostrom1yyca08xqdgvjz0psg56z67ejh9xms6l436u8y5...,0,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,NTN
3,bostrom,,bostrom1yw4xvtc43me9scqfr2jr2gzvcxd3a9y4eq7gau...,0,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,NTN
4,bostrom,,bostrom1cnuw3f076wgdyahssdkd0g3nr96ckq8cwa2mh0...,0,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,SKI
...,...,...,...,...,...,...
101,bostrom,,bostrom1s5g4rehkansq4pamj42usczezqkchjke4j2kk7...,0,Qma9xsv2iWdecCQtrkQoiXwQhd7dWvJ6175kjmjhgSKfcQ,TTT
102,bostrom,,bostrom1seeld9pqq0nxp3yt3wded4jf476y8vz77qmjj5...,0,QmefkBJpLP8nVZknst77mLFznRCgvn1UnSMAU1zfXgJe6d,ABC
103,bostrom,,bostrom10stecvwvxzkh3ah7y6j0kcwe3chmrg37zevjgj...,10,QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH,AAAC
104,bostrom,,bostrom1w4dff5myjyzymk8tkpjrzj6gnv352hcdpt2dsz...,6,QmRbEoWS3jPxYkb2yzHv7Sio3tr3sU1oQArmhBAGuXaVZA,WGUT


In [11]:
cw20_json = cw20_df[['chain_id', 'channel_id', 'decimals', 'logo', 'ticker', 'contract']].to_json(orient='records')
with open('data/tokens_cw20.json', 'w') as file:
    file.write(cw20_json)

## initialize contract

In [6]:
if INIT_CONTRACT:
    contract_address = instantiate_contract(
        init_query=f'''{{"owner":"{WALLET_ADDRESS}"}}''',
        contract_code_id='19',
        contract_label='cw20 token list',
        contract_admin='bostrom1xszmhkfjs3s00z2nvtn7evqxw3dtus6yr8e4pw',
        from_address=WALLET_ADDRESS,
        display_data=True)
contract_address

'bostrom1uay5ex5z547dw7mhw4puwcg998zu474czftlv5s9qpmywl2umaus4vd6x3'

## upload data

In [7]:
def hub_upload(
        tx_unsigned_file_name: str,
        items_for_upload: list[dict],
        contract_address: str,
        required_keys: list[str],
        sender: Optional[Union[str, AccAddress]] = None,
        lcd_client: LCDClient = LCD_CLIENT,
        fee_denom: str = 'boot',
        gas: int = 5_000_000,
        wallet: Optional[Wallet] = None,
        sign_and_broadcast_tx: bool = False,
        memo='add entry') -> dict:
    _execute_msgs = []
    for _item_for_upload in items_for_upload:
        assert all(_item in _item_for_upload.keys() for _item in required_keys)
        _execute_msg = {'CreateEntry': _item_for_upload}
        _execute_msgs.append(_execute_msg)
    _tx_unsigned = execute_contract(
        execute_msgs=_execute_msgs,
        contract_address=contract_address,
        sender=sender,
        wallet=wallet,
        lcd_client=lcd_client,
        fee_denom=fee_denom,
        gas=gas,
        sign_and_broadcast_tx=sign_and_broadcast_tx,
        memo=memo
    )
    with open(tx_unsigned_file_name, 'w') as _tx_unsigned_file:
        _tx_unsigned_file.write(json.dumps(_tx_unsigned.to_data()))
    return _tx_unsigned.to_data()

In [8]:
contract_name = 'tokens_cw20'
with open(f'data/{contract_name}.json', 'r') as file:
    data_for_upload = json.load(file)
tx_unsigned_data = hub_upload(
    tx_unsigned_file_name=f'txs/tx_{contract_name}_upload_unsigned.json',
    items_for_upload=data_for_upload[1:],
    contract_address=contract_address,
    sender=WALLET_ADDRESS,
    wallet=WALLET,
    required_keys=['chain_id', 'channel_id', 'contract', 'decimals', 'logo', 'ticker'],
    gas=24_000_000,
    sign_and_broadcast_tx=False,
    memo='upload cw20 token list'
)
tx_unsigned_data

{'body': {'messages': [{'sender': 'bostrom1cj8j6pc3nda8v708j3s4a6gq2jrnue7j857m9t',
    'contract': 'bostrom1uay5ex5z547dw7mhw4puwcg998zu474czftlv5s9qpmywl2umaus4vd6x3',
    'msg': {'CreateEntry': {'chain_id': 'bostrom',
      'channel_id': '',
      'decimals': '0',
      'logo': 'QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH',
      'ticker': 'NTN',
      'contract': 'bostrom1suhgf5svhu4usrurvxzlgn54ksxmn8gljarjtxqnapv8kjnp4nrs3mewwz'}},
    'funds': [],
    '@type': '/cosmwasm.wasm.v1.MsgExecuteContract'},
   {'sender': 'bostrom1cj8j6pc3nda8v708j3s4a6gq2jrnue7j857m9t',
    'contract': 'bostrom1uay5ex5z547dw7mhw4puwcg998zu474czftlv5s9qpmywl2umaus4vd6x3',
    'msg': {'CreateEntry': {'chain_id': 'bostrom',
      'channel_id': '',
      'decimals': '0',
      'logo': 'QmbFMke1KXqnYyBBWxB74N4c5SBnJMVAiMNRcGu6x1AwQH',
      'ticker': 'NTN',
      'contract': 'bostrom1yyca08xqdgvjz0psg56z67ejh9xms6l436u8y58m82npdqqhmmtquldtvc'}},
    'funds': [],
    '@type': '/cosmwasm.wasm.v1.MsgExecute